Chapter 7: Classes
=============================

**[Arthur Goldberg](https://www.mountsinai.org/profiles/arthur-p-goldberg)**

This notebook was created for the [Biomedical Software Engineering](https://learn.mssm.edu/webapps/blackboard/content/listContentEditable.jsp?content_id=_448512_1&course_id=_5776_1 "Biomedical Software Engineering Blackboard site") course at the [Mount Sinai School of Medicine](https://icahn.mssm.edu/).


A `class` combines code and data.

This example declares and uses a class called `Person`. Note the following:
+ class definitions begin with the keyword `class`
+ every Python class *inherits* functionality from at least one *base* class; in this example the base class is Python's built-in `object` class; we will return to inheritance later
+ a class definition may contain arbitrarily many functions, which are called *methods*
+ the name of a class becomes a `constructor` that creates and returns *instances* (individual *objects*) of the class
+ a class' `__init__` function defines the constructor's initialization behavior -- its parameters and code
+ in a class' methods, the first parameter is, by convention, `self`, which refers to the class' instance; more on methods later
+ the construct `self.name` refers to an *attribute* of a class; attributes can be written and read, and store data and functions (which are data themselves)
+ by convention ([PEP8](https://www.python.org/dev/peps/pep-0008/?)) class names are written in `CamelCase` style, whereas methods and attributes are written in `lower_case_with_underscore` style

Also note that executing a Jupyter cell prints the return value of the last statement in the cell.

In [5]:
class Person(object):
    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name

    def whole_name(self):
        return self.first_name + ' ' + self.last_name

arthur_goldberg = Person('Arthur', 'Goldberg')
arthur_goldberg.whole_name()

'Arthur Goldberg'

More observations:
+ arbitrarily many instances can be created
+ user-defined classes are `class` instances, just like built-in classes
+ insances of user-defined classes can be used just like instances of built-in classes
+ printing (converting to a string and outputting to standard out) a user defined class is not very informative

In [10]:
dean = Person('Dennis', 'Charney')
print(dean.whole_name())

# what are Person and list
print('Person:', Person)
print('list:', list)   # some built-in classes do not follow CamelCase

# a dict of Persons
people = {
    'arthur': arthur_goldberg,
    'dennis': dean
}
for first_name, person in people.items():
    print('first_name:', first_name, '; person.whole_name():', person.whole_name(),
          '; person.last_name:', person.last_name)
print('dean:', dean)

Dennis Charney
Person: <class '__main__.Person'>
list: <class 'list'>
first_name: arthur ; person.whole_name(): Arthur Goldberg ; person.last_name: Goldberg
first_name: dennis ; person.whole_name(): Dennis Charney ; person.last_name: Charney
dean: <__main__.Person object at 0x7faee868b470>


Let's make a more interesting `Person`. New features and approaches:
+ it's important to validate input; errors should be reported by raising `exceptions`; coming soon
+ a method decorated by `@staticmethod` like `initial_capital` is a method that doesn't access the instance in `self`; instead, it's just a function in a class; read about `decorators` please
+ a method decorated by `@classmethod` like `validate_names` is a method that takes a reference to its `class` as its first argument
+ what does `map` in `validate_age_dob` do?
+ `validate_age_dob` returns a tuple of `(error, value)`, which is easy to use
+ when I take code from stackoverflow I cite it
+ `Person` is still vulnerable to bad input; what would `Person('', 'last', 0, '2000-01-01')` or `Person('first', 'last', 0, '2000-01-X')` do?

In [33]:
import datetime # date and time handling package in the standard library

class Person(object):
    def __init__(self, first_name, last_name, age, date_of_birth):
        # date_of_birth in YYYY-MM-DD format
        errors = self.validate_names([first_name, last_name])
        if errors:
            print(errors)   # better to raise an exception; coming later
            return
        self.first_name = first_name
        self.last_name = last_name

        error, dob_instance = self.validate_age_dob(age, date_of_birth)
        if error:   # better to raise an exception
            print('age and DOB are invalid; {} {} is'.format(first_name, last_name), error)
            return
        self.date_of_birth = dob_instance

    @staticmethod
    def initial_capital(name):
        return name[:1].isupper()

    @classmethod
    def validate_names(cls, names):
        errors = []
        for name in names:
            if not cls.initial_capital(name):
                errors.append("error: '{}' is missing an initial capital letter".format(name))
        return errors

    @classmethod
    def validate_age_dob(cls, age, dob):
        year, month, day = map(int, dob.split('-'))
        dob_instance = datetime.date(year, month, day)
        actual_age = datetime.date.today().year - dob_instance.year
        if actual_age < age:
            return ('actually younger', None)
        if age < actual_age:
            return ('actually older', None)
        return (False, dob_instance)

    def age(self):
        today = datetime.date.today()
        # from https://stackoverflow.com/a/9754466/509882:
        return today.year - self.date_of_birth.year - ((today.month, today.day) < (self.date_of_birth.month, self.date_of_birth.day))

    def whole_name(self):
        return self.first_name + ' ' + self.last_name

    def __str__(self):
        attributes = []
        attributes.append('name: ' + self.whole_name())
        attributes.append('age: {}'.format(self.age()))
        return '; '.join(attributes)

Person('not capitalized', 'Goldberg', 0, '2000-01-01')
Person('X', 'Y', 50, '1965-01-01')
prof = Person('Arthur', 'Goldberg', 63, '1955-08-17')
print(prof)

["error: 'not capitalized' is missing an initial capital letter"]
age and DOB are invalid; X Y is actually older
name: Arthur Goldberg; age: 63
